---
title: "Three.js - 3D Animations in the Browser"
author: "Vahram Poghosyan"
date: "2024-12-26"
categories: ["Three.js", "Visualization", "JavaScript"]
format:
  html:
    css: ./css/three-js-demo.css
    code-fold: false
toc-depth: 4
jupyter: python3
highlight-style: github
include-after-body:
  text: |
    <script type="application/javascript" src="../../../javascript/light-dark.js"></script>
    <script type="importmap">
      {
          "imports": {
              "three": "https://cdn.jsdelivr.net/npm/three@0.173.0/+esm",
              "OBJLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/OBJLoader.js"
          }
      }
    </script>
    <script type="module" src="./javascript/three-js-demo.js"></script>
    <script type="module" src="./javascript/three-js-grid-demo.js"></script>
    <script type="module" src="./javascript/three-js-custom-object-demo.js"></script>
---

# Introduction

In this post we use [Three.js](https://threejs.org/) external scripts to render 3D scenes inside this Jupyter notebook. 

This post will be very similar, in terms of its stack, to the [D3.js interactive US map posy](../d3_in_jupyter_with_deno/d3_js_in_jupyter_with_deno.ipynb).

In *that* post, we learned that Canvas and SVG are two ways in which we can display complex graphics inside a web browser. We already explored SVG graphics in [D3.js interactive US map](../d3_in_jupyter_with_deno/d3_js_in_jupyter_with_deno.ipynb). It's worth noting that a lot of the same functionality could've been replicated using the HTML Canvas element instead of SVG (which we ultimately chose for its superior interactive capabilities) -- the article mentions one way to do that by using [skia-canvas](https://github.com/samizdatco/skia-canvas). 

We ended up using [linkedom](https://github.com/WebReflection/linkedom#readme) to add a DOM API on top of the Deno environment. 

In fact, what I realized later on is that we could've simply used simple Markdown inside our Jupyter notebook to create the `<svg>` element without the need to introduce a DOM API on top of a browser-less JavaScript environment. In that case, we could have fetched the `TopoJSON` data and constructed the map inside our external scripts (which we ended up reserving *only* for the D3 animations). These scripts are run by Quarto only *after* the page has been rendered to the browser, so we could easily reference the DOM by `class` or `id`. Crucially, the external scripts are meant to run *inside the browser* (as opposed to being pre-computed in the browser-less Deno environment) where they're able to leverage the existing DOM API provided by the browser's engine (e.g. `document.getElementById`). 

The [D3.js US map post](../d3_in_jupyter_with_deno/d3_js_in_jupyter_with_deno.ipynb) shows a *mixture* of approaches that still works. However, in *this* post we will simplify things a little by sticking with one approach only. We will not use either `skia-canvas` or `linkedom` for generating the `<canvas>` (or `<svg>`) element. Instead, we will add a `<canvas>` element below this very Jupyter notebook cell as plain HTML.

This approach effectively means we can also do away with Deno itself. The notebook does not to pre-compute any JavaScript, it can just serve as the DOM scaffolding for the external scrips to hook into. 

# Three.js

[Three.js](https://threejs.org/) is a library for drawing 3D graphics in the browser using JavaScript and [WebGL](https://get.webgl.org/) (see [wiki](https://www.khronos.org/webgl/wiki/Main_Page)). 

WebGL runs in an HTML Canvas (i.e. `<canvas>`). Canvas and SVG are two ways in which we can display complex graphics inside a browser. We already explored SVG graphics in the [D3.js interactive US Map](../d3_in_jupyter_with_deno/d3_js_in_jupyter_with_deno.ipynb) post. It's worth noting that a lot of the same functionality could've been achieved by using the HTML Canvas instead. The article mentions one way to do that by using [skia-canvas](https://github.com/samizdatco/skia-canvas). 

In that post, we ended up using a combination of [linkedom](https://github.com/WebReflection/linkedom#readme) to mock a DOM API on the Deno environment. In fact, what I realized later on is that we could've simply used the Markdown inside our Jupyter notebook to create the `<svg>` element. Then, we could construct the rest of the map inside our external scripts (which we used only for animations). These scripts are meant to run only *after* the page has been rendered to the browser, so we could easily reference the DOM by `class` or `id`. Crucially, the external scripts run *in the browser* (as opposed to being pre-computed in the browser-less Deno environment). In the browser they are able to leverage the DOM API provided by the browser's engine (e.g. `document.getElementById`). 

The D3 post shows a mixture of approaches that works. However, in this post we will simplify things by sticking to one approach. We will not use either `skia-canvas` or `linkedom` for generating the `<canvas>` (or `<svg>`) element. Instead, I'll include one below this very Jupyter notebook cell as plain HTML.

This approach effectively means we can also do away with Deno. The notebook itself need not pre-compute any JavaScript. 

## Understanding How Three.js Works

But the HTML Canvas is not the focus of this post. 

From the WebGL wiki: 

> WebGL is a DOM API, which means that it can be used from any DOM-compatible language: e.g. JavaScript

Three.js is a library that provides conveniences in JavaScript that abstract much of this DOM API calls. Note that Three.js isn't suitable for modelling purposes, for that we can use [Blender](https://www.blender.org/) or a number of other closed-source 3D modelling applications. We can even download or purchase third-party models from vendors.

# 3D Scenes in General

In *any* 3D scene, be it in the web browser, inside a game engine, in a movie, in a 3D modelling application, etc. there are a bunch of **geometries**, or **shapes** (or **meshes**) that get styled with **textures** (which can range from a complex mapping, like a **shader** animation, to a simple [raster](https://en.wikipedia.org/wiki/Raster_graphics) image).

A scene will also have one or more **light sources**, and a **camera** to *serve* the scene in some perspective.

# First Canvas

Let's create a `<canvas>` with `id="three-d-canvas"`.

We can do it either using raw markup below this very cell, or by using the `IPython.display` module which allows us to display rich representations of objects in a Jupyter notebook (much like the `Deno.jupyter.display` module we saw in the [D3.js US map post](../d3_in_jupyter_with_deno/d3_js_in_jupyter_with_deno.ipynb)).

Using `IPython.display` is more reliable.

In [13]:
from IPython.display import display, HTML
display(HTML("<canvas id='three-d-canvas'></canvas>"))

The code to produce the above output is this:

```javascript
import * as three from 'https://cdn.jsdelivr.net/npm/three@0.173.0/+esm'

const scene = new three.Scene();

const body = document.getElementById("quarto-document-content");
const bodyWidth = body.clientWidth;
const bodyHeight = 600;

const canvas = document.getElementById("three-d-canvas")

const camera = new three.PerspectiveCamera(75, bodyWidth / bodyHeight, 0.1, 1000);

const renderer = new three.WebGLRenderer({
    canvas: canvas
});

renderer.setPixelRatio( window.devicePixelRatio );
renderer.setSize( bodyWidth, bodyHeight );
camera.position.setZ(30);

renderer.render(scene, camera);

const geometry = new three.TorusGeometry(10,3,16,100);
const material = new three.MeshBasicMaterial({ color: 0xFF6347, wireframe: true });
const torus = new three.Mesh(geometry, material);

scene.add(torus);

function animate() {
    requestAnimationFrame(animate);

    torus.rotation.x += 0.01;
    torus.rotation.y += 0.005;
    torus.rotation.z += 0.01;

    renderer.render(scene, camera);
}

animate();
```

# Grid View

To display multiple 3D `Scenes` in a grid, we can simply use CSS-grid.

First we lay out the markup inside the Jupyter note:

```html
<div class="three-d-grid-container">
    <div class="three-d-grid-item"><canvas id="three-d-canvas-1"></canvas></div>
    <div class="three-d-grid-item"><canvas id="three-d-canvas-2"></canvas></div>
    <div class="three-d-grid-item"><canvas id="three-d-canvas-3"></canvas></div>
    <div class="three-d-grid-item"><canvas id="three-d-canvas-4"></canvas></div>
</div>
```

Then, we include the stylesheet in the post subdirectory and use something like Quarto's `include-after-body` (as we do with the external script(s)):

```yaml
include-after-body:
    <script type="module" src="./javascript/three-js-demo.js"></script>
```

It turns out that there's a supported way of including CSS inside a Quarto post. It's done using the `format.html` flag as follows:

```yaml
format:
  html:
    css: ./css/three-js-demo.css
```

The file `three-js-demo.css` should contain: 

```css
#three-d-grid-container {
    display: grid;
    grid-template-columns: repeat(2, 1fr);
    grid-template-rows: repeat(2, 1fr);
    gap: 10px;
    width: 100%;
    height: 50%;
}

.three-d-grid-item {
    display: flex;
}
```

In [14]:
from IPython.display import display, HTML
display(HTML(
    """
    <div id='three-d-grid-container'>
        <div class="three-d-grid-item" id="three-d-grid-item-1"><canvas id="three-d-canvas-1"></canvas></div>
        <div class="three-d-grid-item" id="three-d-grid-item-2"><canvas id="three-d-canvas-2"></canvas></div>
        <div class="three-d-grid-item" id="three-d-grid-item-3"><canvas id="three-d-canvas-3"></canvas></div>
        <div class="three-d-grid-item" id="three-d-grid-item-4"><canvas id="three-d-canvas-4"></canvas></div>
    </div>
    """
))

<br/>
Now, inside the external script, we can reference the various `canvas` elements by `id`.

# Loading Custom Objects

Quarto won't serve `.obj` files (at least by default), so we can just commit the object file to the remote repository of this digital garden and use the `raw` Github link almost as a CDN.

We will also need the `OBJLoader` Three.js add-on. This can be grabbed from a CDN as well. We can even include it in the `include-after-body.text` article front matter *before* the script file itself as:

```yaml
<script type="importmap">
    {
        "imports": {
            "three": "https://cdn.jsdelivr.net/npm/three@0.173.0/+esm",
            "OBJLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/OBJLoader.js"
        }
    }
</script>
```
And then, within the script, import as follows:

```js
import * as three from 'three';
import { OBJLoader } from 'OBJLoader';
```

Here's the full code to render a rubber ducky who will help us debug our code going forward. I downloaded this model from [Sketchfab](https://sketchfab.com/3d-models/rubber-duck-ecb9ce9ff973406398ee56e391f9c902) which hosts many such free objects.

```javascript
import * as three from 'https://cdn.jsdelivr.net/npm/three@0.173.0/build/three.module.js';
import { OBJLoader } from 'https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/OBJLoader.js';

const body = document.getElementById("quarto-document-content");
const bodyWidth = body.clientWidth;
const bodyHeight = 600;

const canvas = document.getElementById("three-d-canvas");
const scene = new three.Scene();
const camera = new three.PerspectiveCamera(75, bodyWidth / bodyHeight, 0.1, 1000);
const renderer = new three.WebGLRenderer({
    canvas: canvas5
});
renderer.setPixelRatio(window.devicePixelRatio);
renderer.setSize(bodyWidth, bodyHeight);
camera.position.setZ(0);
camera.position.setY(20);

function init(geometry) {

    const material = new three.MeshNormalMaterial();
    const pottedPlant = new three.Mesh(geometry, material)

    pottedPlant.scale.set(5, 5, 5);
    pottedPlant.position.set(0, 15, -20);
    scene.add(pottedPlant);

    function animate() {
        requestAnimationFrame(animate);
        pottedPlant.rotation.y += 0.01;
        renderer.render(scene, camera);
    }
    animate();
}

const loader = new OBJLoader();
loader.load(
    // URL to the OBJ file
    'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/models/rubber_duck.obj',
    // Called when the resource is loaded
    (object) => {

        const duck = object.children[0].geometry;
        const beak = object.children[1].geometry;
        const wings = object.children[2].geometry;
        const eyes = object.children[3].geometry;

        init(duck);
        init(beak);
        init(wings);
        init(eyes);
    }, 
    // Called while loading is progressing
    (xhr) => {
        console.log("Loading object...");
    },
    // Called when loading has errors
    (error) => {
        console.error('An error occured:', error);
    }
);
```
Note that `OBJLoader` takes the URL of the object followed by a callback function that gets triggered by the object fully loading. The geometries of the object are in `object.children[n].geometry`. Each child is a part of the object (for example the eyes, wings, and nose of a duck). The `Mesh` constructor takes these `geometry` data passed to it through the callback function `init`. This is a common pattern in Three.js.

Let's create another canvas to render the rubber ducky.

In [15]:
from IPython.display import display, HTML
display(HTML("<canvas id='#three-d-canvas-5'></canvas>"))